# **PYSPARK INTERVIEW QUESTIONS - ANUJ GOGATE**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

**Q1 While ingesting customer data from an external source, you notice duplicate entries. How would you remove duplicates and retain only the latest entry based on a timestamp column?**

In [0]:
data = [("101", "2023-12-01", 100), ("101", "2023-12-02", 150), 
        ("102", "2023-12-01", 200), ("102", "2023-12-02", 250)]
columns = ["product_id", "date", "sales"]

df = spark.createDataFrame(data, columns)
df.createOrReplaceTempView('orders')
df.display()

product_id,date,sales
101,2023-12-01,100
101,2023-12-02,150
102,2023-12-01,200
102,2023-12-02,250


In [0]:
spark.sql("""
          with table_cte1 as 
          (select product_id, max(date) as latest_date
          from orders
          group by product_id
          )
          select * from table_cte1
          """).show()

+----------+-----------+
|product_id|latest_date|
+----------+-----------+
|       101| 2023-12-02|
|       102| 2023-12-02|
+----------+-----------+



In [0]:
df.drop

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-6791606966031709>, line 1
----> 1 data.show()

AttributeError: 'list' object has no attribute 'show'

**Solution**

In [0]:
spec = Window.partitionBy('product_id').orderBy('date')

df1 = df.withColumn('row_check',row_number().over(spec))

df2 = df1.filter(col('row_check') == 2).drop('row_check')

df2.display()

product_id,date,sales
101,2023-12-02,150
102,2023-12-02,250


**2. While processing data from multiple files with inconsistent schemas, you need to merge them into a single DataFrame. How would you handle this inconsistency in PySpark?**

**Solution**

**4. You are working with a real-time data pipeline, and you notice missing values in your streaming data Column - Category. How would you handle null or missing values in such a scenario?**

**df_stream = spark.readStream.schema("id INT, value STRING").csv("path/to/stream")**

**5. You need to calculate the total number of actions performed by users in a system. How would you calculate the top 5 most active users based on this information?**

In [0]:
data = [("user1", 5), ("user2", 8), ("user3", 2), ("user4", 10), ("user2", 3)]
columns = ["user_id", "actions"]

df = spark.createDataFrame(data, columns)
df.display()

user_id,actions
user1,5
user2,8
user3,2
user4,10
user2,3


In [0]:
df1 = df.groupBy('user_id').agg(sum('actions').alias('actions')).orderBy(desc('actions')).show()

+-------+-------+
|user_id|actions|
+-------+-------+
|  user2|     11|
|  user4|     10|
|  user1|      5|
|  user3|      2|
+-------+-------+



In [0]:
df1 = df.groupBy('user_id').agg(sum('actions').alias('actions')).orderBy(desc(col('actions')))

df1.show()

+-------+-------+
|user_id|actions|
+-------+-------+
|  user2|     11|
|  user4|     10|
|  user1|      5|
|  user3|      2|
+-------+-------+



**6. While processing sales transaction data, you need to identify the most recent transaction for each customer. How would you approach this task?**

In [0]:
data = [("cust1", "2023-12-01", 100), ("cust2", "2023-12-02", 150),
        ("cust1", "2023-12-03", 200), ("cust2", "2023-12-04", 250)]
columns = ["customer_id", "transaction_date", "sales"]
df = spark.createDataFrame(data, columns)
df.display()

customer_id,transaction_date,sales
cust1,2023-12-01,100
cust2,2023-12-02,150
cust1,2023-12-03,200
cust2,2023-12-04,250


In [0]:
spec = Window.partitionBy('customer_id').orderBy(desc('transaction_date'))

df1 = df.withColumn('row_chk',row_number().over(spec)).filter(col('row_chk') == 1).drop('row_chk')

display(df1)

**7. You need to identify customers who haven’t made any purchases in the last 30 days. How would you filter such customers?**

In [0]:
data = [("cust1", "2025-12-01"), ("cust2", "2024-11-20"), ("cust3", "2024-11-25")]
columns = ["customer_id", "last_purchase_date"]

df = spark.createDataFrame(data, columns)

df.display()

customer_id,last_purchase_date
cust1,2025-12-01
cust2,2024-11-20
cust3,2024-11-25


In [0]:
df1 = df.withColumn('last_purchase_date',col('last_purchase_date').cast('date'))

df2 = df1.withColumn('gap',datediff(current_date(), 'last_purchase_date')).filter(col('gap') > 30).drop('last_purchase_date', 'gap')

df2.display()

customer_id
cust2
cust3


**8. While analyzing customer reviews, you need to identify the most frequently used words in the feedback. How would you implement this?**

In [0]:
data = [("customer1", "The product is great"), ("customer2", "Great product, fast delivery"), ("customer3", "Not bad, could be better")]
columns = ["customer_id", "feedback"]

df = spark.createDataFrame(data, columns)

df.createOrReplaceTempView('customers')

df.display()

spark.sql("select * from customers").show(truncate=False)

customer_id,feedback
customer1,The product is great
customer2,"Great product, fast delivery"
customer3,"Not bad, could be better"


+-----------+----------------------------+
|customer_id|feedback                    |
+-----------+----------------------------+
|customer1  |The product is great        |
|customer2  |Great product, fast delivery|
|customer3  |Not bad, could be better    |
+-----------+----------------------------+



In [0]:
df1 = df.withColumn('words_split',split(col('feedback'), ' ')) \
    .withColumn('new',explode(col('words_split'))) \
    .withColumn('new', regexp_replace(col('new'), '[^A-Za-z]', '')) \
    .groupBy('new') \
    .count() \
    .orderBy(desc('count')) \
    .drop('count') \
    .limit(1)

most_frequent_word = df1.collect()[0][0]
print(f" The most frequently used words in the feedback is only {most_frequent_word}.")

 The most frequently used words in the feedback is only product.


In [0]:
df1 = spark.sql("""
          with test_cte1 as
          (select *, split(feedback, ' ') as words_splt
          from customers),

          test_cte2 as
          (select *, explode(words_splt) as new
          from test_cte1),

          test_cte3 as
          (select *, lower(regexp_replace(new, '[^A-Za-z]', '')) as new_col
          from test_cte2),

          test_cte4 as
          (select new_col, count(*) as total_cnt
          from test_cte3 group by new_col),

          max_cnt_cte as
          (select max(total_cnt) as max_cnt
          from test_cte4
          )

          select new_col from 
          (select new_col, total_cnt 
          from test_cte4
          join max_cnt_cte 
          on test_cte4.total_cnt = max_cnt_cte.max_cnt)
          
          
          """)
most_frequent_words = [x['new_col'] for x in df1.collect()]

most_frequent_words_str = ' and '.join(most_frequent_words)

print(f" The most frequently used words in the feedback is {most_frequent_words_str}.")

 The most frequently used words in the feedback is product and great.


**9. You need to calculate the cumulative sum of sales over time for each product. How would you approach this?**

In [0]:
data = [("product1", "2023-12-01", 100), ("product2", "2023-12-02", 200),
        ("product1", "2023-12-03", 150), ("product2", "2023-12-04", 250)]
columns = ["product_id", "date", "sales"]
df = spark.createDataFrame(data, columns)
df.createOrReplaceTempView("orders")
df.display()

product_id,date,sales
product1,2023-12-01,100
product2,2023-12-02,200
product1,2023-12-03,150
product2,2023-12-04,250


In [0]:
spec = Window.partitionBy('product_id').orderBy('date')

df1 = df.withColumn('cum_sales', lag('sales', 1, 0).over(spec)) \
        .withColumn('cum_sales', col('sales') + col('cum_sales'))

df1.display()


product_id,date,sales,cum_sales
product1,2023-12-01,100,100
product1,2023-12-03,150,250
product2,2023-12-02,200,200
product2,2023-12-04,250,450


In [0]:
spark.sql("""
          select product_id, date, sales, 
          (sales + previous_sales) as previous_sales
          from
          (select *, 
          lag(sales, 1, 0) over (partition by product_id order by date) as previous_sales
          from orders)
          """).show()

+----------+----------+-----+--------------+
|product_id|      date|sales|previous_sales|
+----------+----------+-----+--------------+
|  product1|2023-12-01|  100|           100|
|  product1|2023-12-03|  150|           250|
|  product2|2023-12-02|  200|           200|
|  product2|2023-12-04|  250|           450|
+----------+----------+-----+--------------+



**10. While preparing a data pipeline, you notice some duplicate rows in a dataset. How would you remove the duplicates without affecting the original order?**

In [0]:
data = [("John", 25), ("Jane", 30), ("John", 25), ("Alice", 22)]
columns = ["name", "age"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df1 = df.dropDuplicates(subset=["name"]).orderBy('name')

display(df1)

**11. You are working with user activity data and need to calculate the average session duration per user. How would you implement this?**

In [0]:
data = [("user1", "2023-12-01", 50), ("user1", "2023-12-02", 60), 
        ("user2", "2023-12-01", 45), ("user2", "2023-12-03", 75)]
columns = ["user_id", "session_date", "duration"]
df = spark.createDataFrame(data, columns)

df.display()

In [0]:
df1 = df.groupBy('user_id').agg(avg('duration').alias('duration'))

df1.display()

**12. While analyzing sales data, you need to find the product with the highest sales for each month. How would you accomplish this?**

In [0]:
data = [("product1", "2023-12-01", 100), ("product2", "2023-12-01", 150), 
        ("product1", "2023-12-02", 200), ("product2", "2023-12-02", 250)]
columns = ["product_id", "date", "sales"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df1 = df.withColumn('month',month(to_date(col('date'),'yyyy-MM-dd'))) \
        .groupBy('product_id').agg(sum('sales').alias('high_sales')) \
        .orderBy(col('high_sales').desc()) \
        .limit(1)

df1.display()

**13. You are working with a large Delta table that is frequently updated by multiple users. The data is stored in partitions, and sometimes updates can cause inconsistent reads due to concurrent transactions. How would you ensure ACID compliance and avoid data corruption in PySpark?**

In [0]:
df = spark.read.parquet() #New Data 

from delta.tables import DeltaTable

delta_table = DeltaTable.forPath('path') #Old Data

delta_table.alias('tgt').merge(df.alias('src'), "src.id" == "tgt.id") \
                        .whenNotMatchedInsertAll() \
                        .whenMatchedUpdateAll() \
                        .execute()

**14. You need to process a large dataset stored in PARQUET format and ensure that all columns have the right schema (Almost). How would you do this?**

In [0]:
df = spark.read.format('parquet') \
        .option('inferSchema',True) \
        .load('path')

**15. You are reading a CSV file and need to handle corrupt records gracefully by skipping them. How would you configure this in PySpark?**

In [0]:
df = spark.read.format('csv') \
     .option('mode','DROPMALFORMED') \
     .load('path')

**22. You have a dataset containing the names of employees and their departments. You need to find the department with the most employees.**

In [0]:
data = [("Alice", "HR"), ("Bob", "Finance"), ("Charlie", "HR"), ("David", "Engineering"), ("Eve", "Finance")]
columns = ["employee_name", "department"]

df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df = df.groupBy('department').agg(count('employee_name').alias('total_emp')).orderBy(desc('total_emp')).limit(1).display()

**23. While processing sales data, you need to classify each transaction as either 'High' or 'Low' based on its amount. How would you achieve this using a when condition**

In [0]:
data = [("product1", 100), ("product2", 300), ("product3", 50)]
columns = ["product_id", "sales"]

df = spark.createDataFrame(data, columns)
df.display()
df.createOrReplaceTempView('sales')

In [0]:
df1 = df.withColumn('tag',when(col('sales') >= 100, 'high').otherwise('low'))

df1.display()

In [0]:
spark.sql("""
select *, case 
when sales >= 100 then 'high' else 'low' end as tag
from sales    
          """).show()

**24. While analyzing a large dataset, you need to create a new column that holds a timestamp of when the record was processed. How would you implement this and what can be the best USE CASE?**

In [0]:
data = [("product1", 100), ("product2", 200), ("product3", 300)]
columns = ["product_id", "sales"]

df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df1 = df.withColumn('date_processed', current_timestamp())

display(df1)

**25. You need to register this PySpark DataFrame as a temporary SQL object and run a query on it. How would you achieve this?**

In [0]:
data = [("product1", 100), ("product2", 200), ("product3", 300)]
columns = ["product_id", "sales"]

df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df.createOrReplaceTempView('sales')

spark.sql("select * from sales").show()

**26. You need to register this PySpark DataFrame as a temporary SQL object and run a query on it (FROM DIFFERENT NOTEBOOKS AS WELL)?**

In [0]:
df.createOrReplaceGlobalTempView('global_temp')

spark.sql('select * from global_temp.global_temp').show()

**27. You need to query data from a PySpark DataFrame using SQL, but the data includes a nested structure. How would you flatten the data for easier querying?**

In [0]:
data = [("product1", {"price": 100, "quantity": 2}), 
        ("product2", {"price": 200, "quantity": 3})]
columns = ["product_id", "product_info"]

df = spark.createDataFrame(data, columns)
df.display()

df.createOrReplaceTempView("Sales")

In [0]:
df.select('product_id','product_info.price','product_info.quantity').createOrReplaceTempView('sales')

spark.sql('select * from sales').show()

**28. You are ingesting data from an external API in JSON format where the schema is inconsistent. How would you handle this situation to ensure a robust pipeline?**

In [0]:
df = spark.read.format('json').option('mergeSchema',True).load('path')

**29. While reading data from Parquet, you need to optimize performance by partitioning the data based on a column. How would you implement this?**

In [0]:
df.write.format('parquet').mode('overwrite').partitionBy('col_name').save('path')

**30. You are working with a large dataset in Parquet format and need to ensure that the data is written in an optimized manner with proper compression. How would you accomplish this?**

In [0]:
df.write.format('parquet').mode('overwrite').option('compression','schema').partitionBy('col_name').save('path')

**31. Your company uses a large-scale data pipeline that reads from Delta tables and processes data using complex aggregations. However, performance is becoming an issue due to the growing dataset size. How would you optimize the performance of the pipeline?**

In [0]:
spark.sql("optimize table_name zorder by ('col')")

In [0]:
"""
optimize coalesce the large number of partitions to few number of partitions.
zorder helps to sort the data using column like order_id.
Applying both is part of optimization which also helps in data skipping/pruning.
"""

**43. You are processing sales data. Group by product categories and create a list of all product names in each category.**

In [0]:
data = [("Electronics", "Laptop"), ("Electronics", "Smartphone"), ("Furniture", "Chair"), ("Furniture", "Table")]
columns = ["category", "product"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df1 = df.groupBy('category').agg(concat_ws(',',collect_list('product')).alias('product'))

df1.show()

**44. You are analyzing orders. Group by customer IDs and list all unique product IDs each customer purchased.**

In [0]:
data = [(101, "P001"), (101, "P002"), (102, "P001"), (101, "P001")]
columns = ["customer_id", "product_id"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df1 = df.groupBy('customer_id').agg(concat_ws(',',collect_set('product_id')).alias('products'))

df1.display()

**45. For customer records, combine first and last names only if the email address exists.**

In [0]:
data = [("John", "Doe", "john.doe@example.com"), ("Jane", "Smith", None)]
columns = ["first_name", "last_name", "email"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df1 = df.filter(col('email').isNotNull())

df1.display()

**46. You have a DataFrame containing customer IDs and a list of their purchased product IDs. Calculate the number of products each customer has purchased.**

In [0]:
data = [
    (1, ["prod1", "prod2", "prod3"]),
    (2, ["prod4"]),
    (3, ["prod5", "prod6"]),
]
myschema = "customer_id INT ,product_ids array<STRING>"

df = spark.createDataFrame(data, myschema)
df.display()

In [0]:
df1 = df.withColumn('product_ids',explode('product_ids')).groupBy('customer_id').agg(count('product_ids').alias('total_cnt'))

df1.display()

**47. You have employee IDs of varying lengths. Ensure all IDs are 6 characters long by padding with leading zeroes.**

In [0]:
data = [
    ("1",),
    ("123",),
    ("4567",),
]
schema = ["employee_id"]

df = spark.createDataFrame(data, schema)
df.display()

In [0]:
df.withColumn('new',len('employee_id') * 0 )

**48. You need to validate phone numbers by checking if they start with "91"**

In [0]:
data = [
    ("911234567890",),
    ("811234567890",),
    ("912345678901",),
]
schema = ["phone_number"]

df = spark.createDataFrame(data, schema)
df.display()

In [0]:
df1 = df.filter(col('phone_number').like('91%'))

df1.display()

**49. You have a dataset with courses taken by students. Calculate the average number of courses per student.**

In [0]:
data = [
    (1, ["Math", "Science"]),
    (2, ["History"]),
    (3, ["Art", "PE", "Biology"]),
]
schema = ["student_id", "courses"]

df = spark.createDataFrame(data, schema)
df.display()

In [0]:
df1 = df.withColumn('course_size',size('courses')).groupBy('student_id').agg(avg('course_size').alias('course_size'))

df1.display()

**50. You have a dataset with primary and secondary contact numbers. Use the primary number if available; otherwise, use the secondary number.**

In [0]:
data = [
    (None, "1234567890"),
    ("9876543210", None),
    ("7894561230", "4567891230"),
]
schema = ["primary_contact", "secondary_contact"]

df = spark.createDataFrame(data, schema)
df.display()

In [0]:
df1 = df.withColumn('main_number',when(col('primary_contact').isNotNull(),col('primary_contact')).otherwise(col('secondary_contact')))

df1.display()

**51. You are categorizing product codes based on their lengths. If the length is 5, label it as "Standard"; otherwise, label it as "Custom".**

In [0]:
data = [
    ("prod1",),
    ("prd234",),
    ("pr9876",),
]
schema = ["product_code"]

df = spark.createDataFrame(data, schema)
df.display()

In [0]:
df1 = df.withColumn('label',when(length('product_code') == 5,'Standard').otherwise('Custom'))

df1.display()

In [0]:
%sql
-- # CREATE TABLE Employees (
-- #     EmployeeID INT PRIMARY KEY,
-- #     Name VARCHAR(50),
-- #     Salary INT
-- # );
 
INSERT INTO Employees (EmployeeID, Name, Salary) VALUES
(1, 'Alice', 50000),
(2, 'Bob', 60000),
(3, 'Charlie', 70000),
(4, 'David', 60000),
(5, 'Eve', 80000);

num_affected_rows,num_inserted_rows
5,5


####2nd highest salary with distinct

In [0]:
%sql
          with table_cte1 as 
          (select *, dense_rank() over(order by salary desc) as row_chk from Employees)

          select * from table_cte1 where row_chk == 2

EmployeeID,Name,Salary,row_chk
3,Charlie,70000,2
